In [20]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import csv
import requests
from tqdm import tqdm
from pandas.io.sql import SQLTable
import webscrape_pd_scrape_patheos as pscrape
import sys
from passlib.hash import pbkdf2_sha256
try:
    from configparser import ConfigParser
    from configparser import SafeConfigParser
except ImportError:
    from ConfigParser import ConfigParser  # ver. < 3.0
    from ConfigParser import SafeConfigParser

In [32]:
def update_system_status_values(file, section, option, value):
    config = ConfigParser()
    config.read(file)
    cfgfile = open(file, 'w')
    if config.has_section(section) == True:
    #try:
        config.set(section, option, value)
    else:
    #except:
        config.add_section(section)
        config.set(section, option, value)
    config.write(cfgfile)
    cfgfile.close()
    
#update_system_status_values('patheos_blogs_info.ini', 'EVANGELICAL', 'new', '100')

In [35]:
def exists_config_file(file, section, option):
    config = ConfigParser()
    config.read(file)
    if config.has_option(section, option) == True:
        page_number = int(config.get(section, option))
    else:
        page_number = 1
    return page_number

In [23]:
url = 'http://www.patheos.com/blogs'
response = requests.get(url)
soup = BS(response.content, 'html.parser')
#soup.prettify

In [24]:
blog_list = []

for blog in soup.find_all('div', attrs={"class":"related-content clearfix related-content-sm decorated channel-list"}):
    blog_url1 = blog.find('a')
    blog_url2 = blog_url1['href']
    blog_list.append(blog_url2) 

#for i in blog_list:
#    print(i)

In [25]:
blog_lists = []
    
for i in blog_list:
    split_url = i.rsplit("/")
    blog_name = split_url[3]
    blog_lists.append(blog_name)
    
blog_dict = {i:[] for i in blog_lists}
#print(blog_dict)

#print(blog_dict['buddhist-blogs'])

In [ ]:
#for i in blog_dict:
#    print(i.upper())
#    for v in blog_dict[i]:
#        print("    - " + v)

In [ ]:
#for i in blog_dict:
#    print("https://www.patheos.com/" + i)

In [26]:
blog_cat_prefix = "https://www.patheos.com/"

while True:
    i = 0    
    for blog_urls in tqdm(blog_dict, desc='Fetch blog urls for each category'):
        #print(blog_urls)
        if blog_urls:
            query_url = blog_cat_prefix + blog_urls
            subsub_blog = requests.get(query_url)
            #if subsub_blog != "<Response [404]>":
            soup2 = BS(subsub_blog.content, 'html.parser')
            for blog in soup2.find_all('div', attrs={"class":"author-info"}):
                #print(blog)
                for blog_url0 in blog.find_all('div', attrs={"class":"title"}):
                    #print(blog_url0)
                    blog_url1 = blog_url0.find('a')
                    blog_url2 = blog_url1['href']
                    blog_dict[blog_urls].append(blog_url2)
                    #print(blog_url2)
            i = i + 1
        else:
            continue
    else:
        break

Fetch blog urls for each category: 100%|███████████████████████████████████████████████| 20/20 [00:06<00:00,  3.99it/s]


In [38]:
def blog_find_pages(blogname_list, list_name):
    c = 0
    t = 1
    b = 1
    url_list = []
    continue_on = True
    page_prefix = "/page/"
    
    for i in blogname_list:
        #p = 1
        continue_on = True
        blog_name = i.rsplit("/")
        p = int(exists_config_file('patheos_blogs_pages.ini', list_name, blog_name[4]))
        t = t + p
        print("\n" + blog_name[4].upper())
        while continue_on == True:
            blog_page_url = i + page_prefix + str(p)
            #print(blog_page_url)
            url_test = requests.get(blog_page_url)
            if url_test.status_code != 404:
                url_list.append(blog_page_url)
                #print(blog_page_url)
                p = p + 1
                t = t + 1
                #c = c + 1
                continue_on = True
            else:
                #e = e + 1
                p = p - 1
                continue_on = False
            sys.stdout.write('\r' + "category number: " + str(b) + "/" + str(len(blogname_list)) 
                             + " | page(" + str(p) + ") | category total: " + str(t) + ")")
        b = b + 1
        update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
    else:
        return url_list



In [39]:
# For in file testing
keep = True
for i in blog_dict:
    while keep == True:
        print(i)
        name = i
        keep = False
        new_list = blog_find_pages(blog_dict[i], i)

buddhist-blogs

AMERICANBUDDHIST
category number: 1/5 | page(160) | category total: 162)
BODHISATTVAROAD
category number: 2/5 | page(14) | category total: 177)
MONKEYMIND
category number: 3/5 | page(439) | category total: 617)
NOTESFROMTHETEMPLE
category number: 4/5 | page(5) | category total: 623)
WILDFOXZEN
category number: 5/5 | page(84) | category total: 708)

In [ ]:
print(new_list)

In [11]:
# For in file testing
n = 0

for i in blog_dict:
    print(i, len(blog_dict[i]))
    n = n + len(blog_dict[i])

print("Total: " + str(n))

buddhist-blogs 5
catholic-blogs 79
contemplative-blogs 9
evangelical-blogs 79
general-christian-blogs 16
hindu-blogs 4
jewish-blogs 6
latter-day-saint-blogs 17
muslim-blogs 34
new-visions-blogs 34
nonreligious-blogs 64
pagan-blogs 53
progressive-christian-blogs 97
more-voices-blogs 3
sixseeds-family-blogs 23
politics-blue-blogs 3
politics-red-blogs 10
entertainment-blogs 12
faith-and-work-blogs 16
patheos-partner-blogs 2
Total: 566
